In [4]:
import pandas as pd
from pandas import ExcelFile, read_excel, ExcelWriter
from numpy import nonzero, isnan, nan, vectorize, int32
from time import sleep
from datetime import datetime



def set_nameOrigin(db_dict):

    for col in list(db_dict.keys()):
        
        if col == 'Name':
            db_dict[col]['nameOrigin'] = db_dict[col]['Name'].astype(str) + ', ' + db_dict[col]['Origin'].astype(str)
        elif col == 'References': 
            db_dict[col]['authorYear'] = db_dict[col]['Author'].astype(str) + ', ' + db_dict[col]['Year'].astype(str)
        elif col == 'Country':
            db_dict[col]['nameOrigin'] = db_dict[col]['Country'].astype(str) + ', ' + db_dict[col]['City'].astype(str)  
        elif col == 'Region':
            pass
        elif col == 'Spartacus Material':
            db_dict[col]['nameOrigin'] = db_dict[col]['Name'].astype(str) + '; ' + db_dict[col]['Color'].astype(str)# + '; ' + db_dict[col]['Material Type'].astype(str)    
        # Calculate U-values for roof and wall new columns u_value_wall and u_value_roof
        
        elif col == 'Spartacus Surface':
            db_dict[col]['nameOrigin'] = db_dict[col]['Name'].astype(str) + ', ' + db_dict[col]['Origin'].astype(str)
                # Filter rows where Surface is 'Buildings'
        
            buildings = db_dict['Spartacus Surface'][db_dict['Spartacus Surface']['Surface'] == 'Buildings']

            # Calculate resistances and U-values
            for prefix in ['w', 'r']:

                if prefix == 'w':
                    pr = 'wall'
                else:
                    pr = 'roof'
                materials = buildings[[f'{prefix}{i}Material' for i in range(1, 6)]].values
                thicknesses = buildings[[f'{prefix}{i}Thickness' for i in range(1, 6)]].values

                thicknesses[isnan(thicknesses)] = 0

                for i in range(0,5):
                    materials[isnan(materials)] = materials[nonzero(isnan(materials))[0], nonzero(isnan(materials))[1]-1]

                thermal_conductivities = vectorize(lambda x: db_dict['Spartacus Material'].loc[x, 'Thermal Conductivity'])(materials)

                resistances = thicknesses / thermal_conductivities
                resistance_bulk = resistances.sum(axis=1)

                u_values = 1 / resistance_bulk

                db_dict['Spartacus Surface'].loc[buildings.index, f'u_value_{pr}'] = u_values

            # Calculate albedo and emissivity
            for prop in ['Albedo', 'Emissivity']:
                for prefix in ['w', 'r']:
                    if prefix == 'w':
                        pr = 'wall'
                    elif prefix == 'r':
                        pr == 'roof'
                    material_col = f'{prefix}1Material'
                    db_dict['Spartacus Surface'].loc[buildings.index, f'{prop.lower()}_{pr}'] = db_dict['Spartacus Material'].loc[buildings[material_col], prop].values
        
        elif col == 'Profiles':
            # Normalise traffic and energy use profiles to ensure that average of all columns = 1
            normalisation_rows = db_dict[col][(db_dict[col]['Profile Type'] == 'Traffic') | (db_dict[col]['Profile Type'] == 'Energy use')]
            cols = list(range(24))
            normalisation_rows_index = list(normalisation_rows.index)

            # # # Calculate the sum of the values for each row
            sums = db_dict[col].loc[normalisation_rows_index, cols].sum(axis=1)

            # Avoid division by zero by replacing zero sums with NaN
            sums.replace(0, nan, inplace=True)

            # # Calculate the scaling factor to make the sum equal to the number of columns (24)
            scaling_factors = 24 / sums

            # Scale the values
            db_dict[col].loc[normalisation_rows_index, cols] = db_dict[col].loc[normalisation_rows_index, cols].multiply(scaling_factors, axis=0)
            
            # Create unique name
            db_dict[col]['nameOrigin'] = db_dict[col]['Name'].astype(str)  +  ', ' + db_dict[col]['Day'].astype(str) +  ', ' + db_dict[col]['Country'].astype(str) + ', ' + db_dict[col]['City'].astype(str) 

        else:
            # Standard
            db_dict[col]['nameOrigin'] = db_dict[col]['Name'].astype(str) + ', ' + db_dict[col]['Origin'].astype(str)
    
    return db_dict

def read_DB(db_path):
    '''
    function for reading database and parse it to dictionary of dataframes
    nameOrigin is used for indexing and presenting the database entries in a understandable way for the user
    '''
    db_sh = ExcelFile(db_path)
    sheets = db_sh.sheet_names
    db_dict = read_excel(db_path, sheet_name= sheets, index_col= 0)

    db_dict = set_nameOrigin(db_dict)
    db_sh.close() # trying this to close excelfile

    return db_dict


In [ ]:
db_path = 'C:/GitHub/SUEWS_DB_Typology_test/backup_UMEP/suews_database_manager/data/database.xlsx'

db_dict = read_DB(db_path)

In [238]:
# wall
wall = pd.DataFrame()

surface_code = 37240907

surface = db_dict['Spartacus Surface'].loc[surface_code]

insulation = surface['wInsulation']

for i in range(1,6):
    try:
        mat = surface[f'w{str(i)}Material']
        mat = db_dict['Spartacus Material'].loc[mat]
        cp = mat['Thermal Conductivity']
        k = mat['Specific Heat']
        p = mat['Density']
        name = mat['Name']
        dz = surf[f'w{str(i)}Thickness']

        wall.loc[i, 'name'] = name
        wall.loc[i, 'dz'] = dz
        wall.loc[i, 'cp'] = cp
        wall.loc[i, 'k'] = k
        wall.loc[i, 'p'] = p
    except:
        wall.loc[i, 'name'] = nan
        wall.loc[i, 'dz'] = nan
        wall.loc[i, 'cp'] = nan
        wall.loc[i, 'k'] = nan
        wall.loc[i, 'p'] = nan

    if i  == insulation:
        wall.loc[i, 'insulation'] = 'yes'
    else:
        wall.loc[i, 'insulation'] = 'no'

insulation = surface['rInsulation']


roof = pd.DataFrame()
for i in range(1,6):
    try:
        mat = surface[f'r{str(i)}Material']
        mat = db_dict['Spartacus Material'].loc[mat]
        cp = mat['Thermal Conductivity']
        k = mat['Specific Heat']
        p = mat['Density']
        name = mat['Name']
        dz = surf[f'r{str(i)}Thickness']

        roof.loc[i, 'name'] = name
        roof.loc[i, 'dz'] = dz
        roof.loc[i, 'cp'] = cp
        roof.loc[i, 'k'] = k
        roof.loc[i, 'p'] = p
    
    except:
        roof.loc[i, 'name'] = nan
        roof.loc[i, 'dz'] = nan
        roof.loc[i, 'cp'] = nan
        roof.loc[i, 'k'] = nan
        roof.loc[i, 'p'] = nan

    if i  == insulation:
        roof.loc[i, 'insulation'] = 'yes'
    else:
        roof.loc[i, 'insulation'] = 'no'

wall

,name,dz,cp,k,p,insulation
1,"Brick, exposed",0.0700,0.770,1000.0,1750.0,no
2,Polyurethane foam,0.0500,0.025,1400.0,30.0,yes
3,"Brick, protected",0.0700,0.560,1000.0,1750.0,no
4,"Plaster, leightweight",0.0125,0.180,1000.0,600.0,no
5,NaN,NaN,NaN,NaN,NaN,no


In [ ]:
insulation = surface['rInsulation']

for i in range(1,6):
    try:
        mat = surface[f'r{str(i)}Material']
        mat = db_dict['Spartacus Material'].loc[mat]
        cp = mat['Thermal Conductivity']
        k = mat['Specific Heat']
        p = mat['Density']
        name = mat['Name']
        dz = surf[f'r{str(i)}Thickness']

        roof.loc[i, 'name'] = name
        roof.loc[i, 'dz'] = dz
        roof.loc[i, 'cp'] = cp
        roof.loc[i, 'k'] = k
        roof.loc[i, 'p'] = p
    except:
        roof.loc[i, 'name'] = nan
        roof.loc[i, 'dz'] = nan
        roof.loc[i, 'cp'] = nan
        roof.loc[i, 'k'] = nan
        roof.loc[i, 'p'] = nan

    if i  == insulation:
        roof.loc[i, 'insulation'] = 'yes'
    else:
        roof.loc[i, 'insulation'] = 'no'


,name,dz,cp,k,p,insulation
1,Limestone Bright,0.0120,0.930,909.0,2711.0,no
2,"Brick, protected",0.0900,0.560,1000.0,1750.0,yes
3,"Brick, protected",0.0125,0.560,1000.0,1750.0,no
4,Mineral wool (batts),NaN,0.038,1030.0,25.0,no
5,"Plaster, dense",NaN,0.570,1000.0,1300.0,no


In [ ]:
roofwall = ''
rr= horizontal_aggregation(surface_code, roofwall, db_dict, no_rho = True)

rr

L3, only one 3 layer


{'dz': {'value': [0.084, 0.025, 0.03, nan, nan]},
 'k': {'value': [0.25, 0.1, 0.025, nan, nan]},
 'cp': {'value': [822.568, 1000.0, 1005.0, nan, nan]}}

In [ ]:

surface_code = 37249758

surface = db_dict['Spartacus Surface'].loc[surface_code]

roofwall = 'r'


insulation = surface[f'{roofwall}Insulation']
agg_surface = pd.DataFrame()
horizontal_layers_list = []

# Check for how many layers present in the surface
for layer in range(1, 6):
    try:
        mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer}Material']]
        horizontal_layers_list.append(layer)
    except:
        pass

layer1 = horizontal_layers_list[: insulation -1]
layer2 = insulation
layer3 = horizontal_layers_list[insulation:]

# Check if insulation layer is set or not
# IF no insulation
if insulation == 6:
    
    d_list = []
    k_list = []
    rho_list = []
    cp_list = []

    for layer in horizontal_layers_list:
        
        mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer}Material']]
        d_list.append(surface[f'{roofwall}{layer}Thickness'])
        k_list.append(mat['Thermal Conductivity'])
        rho_list.append(mat['Density'])
        cp_list.append(mat['Specific Heat'])

    agg_surface.loc[1,'dz'] = sum(d_list)
    agg_surface.loc[1,'k'] = sum(d_list) / sum(d / k for d, k in zip(d_list, k_list))
    agg_surface.loc[1,'rho'] = sum(d * rho for d, rho in zip(d_list, rho_list)) / sum(d_list)
    agg_surface.loc[1,'cp'] = sum(d * rho * cp for d, rho, cp in zip(d_list, rho_list, cp_list)) / (agg_surface.loc[1,'rho'] * agg_surface.loc[1,'dz'])

    # Fill layer 2-5 with nan
    for layer in range(2,6):
        agg_surface.loc[layer,'dz'] = nan
        agg_surface.loc[layer,'k'] = nan
        agg_surface.loc[layer,'rho'] = nan
        agg_surface.loc[layer,'cp'] = nan

# If insulation Exists. 
else:
    # ------------------------------------- Layer 1 -------------------------------------
    #                                     Outer Layer
    # -----------------------------------------------------------------------------------

    d_list = []
    k_list = []
    rho_list = []
    cp_list = []

    if len(layer1) == 1:
        mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer1[0] }Material']]
        agg_surface.loc[1,'dz'] = surface[f'{roofwall}{layer1[0]}Thickness']
        agg_surface.loc[1,'k'] = mat['Thermal Conductivity']
        agg_surface.loc[1,'rho'] = mat['Density']
        agg_surface.loc[1,'cp'] = mat['Specific Heat']

    else:
        for layer in layer1:
            
            d_list.append(surface[f'{roofwall}{layer}Thickness'])
            mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer}Material']]
            k_list.append(mat['Thermal Conductivity'])
            rho_list.append(mat['Density'])
            cp_list.append(mat['Specific Heat'])

            # print('Outer layer [1]: 1 -',layer)
            agg_surface.loc[1,'dz'] = sum(d_list)
            agg_surface.loc[1,'k'] = sum(d_list) / sum(d / k for d, k in zip(d_list, k_list))
            agg_surface.loc[1,'rho'] = sum(d * k for d, k in zip(d_list, rho_list)) / sum(d_list)
            agg_surface.loc[1,'cp'] = sum(d * rho * cp for d, rho, cp in zip(d_list, rho_list, cp_list)) / (agg_surface.loc[1,'rho'] * agg_surface.loc[1,'dz'])

    # ------------------------------------- Layer 2 -------------------------------------
    #                                   Insulation Layer
    # -----------------------------------------------------------------------------------


    # Just take the values in the layer
    mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer2 }Material']]
    agg_surface.loc[2,'dz'] = surface[f'{roofwall}{layer2}Thickness']
    agg_surface.loc[2,'k'] = mat['Thermal Conductivity']
    agg_surface.loc[2,'rho'] = mat['Density']
    agg_surface.loc[2,'cp'] = mat['Specific Heat']

    # ------------------------------------- Layer 3 -------------------------------------
    #                                     Inner Layer
    # -----------------------------------------------------------------------------------
    
    if len(layer3) > 0:

        if len(layer3) == 1:
            print('L3, only one 3 layer')
            mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer3[0]}Material']]
            agg_surface.loc[3,'dz'] = surface[f'{roofwall}{layer3[0]}Thickness']
            agg_surface.loc[3,'k'] = mat['Thermal Conductivity']
            agg_surface.loc[3,'rho'] = mat['Density']
            agg_surface.loc[3,'cp'] = mat['Specific Heat']

            # Fill 2 inner layers with nan
            for layer in range(4,6):
                agg_surface.loc[layer,'dz'] = nan
                agg_surface.loc[layer,'k'] = nan
                agg_surface.loc[layer,'rho'] = nan
                agg_surface.loc[layer,'cp'] = nan

        else:
            print('L3, more than one 3 layer')

            for layer in layer3:
                d_list = []
                k_list = []
                rho_list = []
                cp_list = []

                for layer in layer3:
                    d_list.append(surface[f'{roofwall}{layer}Thickness'])
                    mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer}Material']]
                    k_list.append(mat['Thermal Conductivity'])
                    rho_list.append(mat['Density'])
                    cp_list.append(mat['Specific Heat'])

                    agg_surface.loc[3,'dz'] = sum(d_list)
                    agg_surface.loc[3,'k'] = sum(d_list) / sum(d / k for d, k in zip(d_list, k_list))
                    agg_surface.loc[3,'rho'] = sum(d * k for d, k in zip(d_list, rho_list)) / sum(d_list)
                    agg_surface.loc[3,'cp'] = sum(d * rho * cp for d, rho, cp in zip(d_list, rho_list, cp_list)) / (agg_surface.loc[3,'rho'] * agg_surface.loc[3,'dz'])

                # Fill 2 inner layers with nan
                for layer in range(4,6):
                    agg_surface.loc[layer,'dz'] = nan
                    agg_surface.loc[layer,'k'] = nan
                    agg_surface.loc[layer,'rho'] = nan
                    agg_surface.loc[layer,'cp'] = nan

    else:
        for layer in range(3,6):
            agg_surface.loc[layer,'dz'] = nan
            agg_surface.loc[layer,'k'] = nan
            agg_surface.loc[layer,'rho'] = nan
            agg_surface.loc[layer,'cp'] = nan

    # -----------------------------------------------------------------------------------

if no_rho == True:
    agg_surface = agg_surface.drop(columns = ['rho'])
    agg_surface = agg_surface.round(3).loc[:,['dz','k','cp']].to_dict()

else:
    agg_surface = agg_surface.round(3).loc[:,['dz','k','cp','rho']].to_dict()

agg_surface = {key: list(value.values()) for key, value in agg_surface.items()}
agg_surface = {key: {'value': value} for key, value in agg_surface.items()}

agg_surface


L3, only one 3 layer


{'dz': {'value': [0.084, 0.025, 0.03, nan, nan]},
 'k': {'value': [0.25, 0.1, 0.025, nan, nan]},
 'cp': {'value': [822.568, 1000.0, 1005.0, nan, nan]}}

[4]

In [ ]:
print(horizontal_layers_list)
print(insulation)

layer1 = horizontal_layers_list[: insulation -1]
layer2 = horizontal_layers_list[insulation]
layer3 = horizontal_layers_list[insulation:]

layer

[1, 2, 3, 4]
3


[4]

In [124]:


def horizontal_aggregation(surface_code, roofwall, db_dict, no_rho = False):

    surface = db_dict['Spartacus Surface'].loc[surface_code]
    insulation = surface[f'{roofwall}Insulation']
    agg_surface = pd.DataFrame()
    horizontal_layers_list = []

    # Check for how many layers present in the surface
    for layer in range(1, 6):
        try:
            mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer}Material']]
            horizontal_layers_list.append(layer)
        except:
            pass

    layer1 = horizontal_layers_list[: insulation -1]
    layer2 = insulation
    layer3 = horizontal_layers_list[insulation:]

    # Check if insulation layer is set or not
    # IF no insulation
    if insulation == 6:
        
        d_list = []
        k_list = []
        rho_list = []
        cp_list = []

        for layer in horizontal_layers_list:
            
            mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer}Material']]
            d_list.append(surface[f'{roofwall}{layer}Thickness'])
            k_list.append(mat['Thermal Conductivity'])
            rho_list.append(mat['Density'])
            cp_list.append(mat['Specific Heat'])

        agg_surface.loc[1,'dz'] = sum(d_list)
        agg_surface.loc[1,'k'] = sum(d_list) / sum(d / k for d, k in zip(d_list, k_list))
        agg_surface.loc[1,'rho'] = sum(d * rho for d, rho in zip(d_list, rho_list)) / sum(d_list)
        agg_surface.loc[1,'cp'] = sum(d * rho * cp for d, rho, cp in zip(d_list, rho_list, cp_list)) / (agg_surface.loc[1,'rho'] * agg_surface.loc[1,'dz'])

        # Fill layer 2-5 with nan
        for layer in range(2,6):
            agg_surface.loc[layer,'dz'] = nan
            agg_surface.loc[layer,'k'] = nan
            agg_surface.loc[layer,'rho'] = nan
            agg_surface.loc[layer,'cp'] = nan

    # If insulation Exists. 
    else:
        # ------------------------------------- Layer 1 -------------------------------------
        #                                     Outer Layer
        # -----------------------------------------------------------------------------------

        d_list = []
        k_list = []
        rho_list = []
        cp_list = []

        if len(layer1) == 1:
            mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer1[0] }Material']]
            agg_surface.loc[1,'dz'] = surface[f'{roofwall}{layer1[0]}Thickness']
            agg_surface.loc[1,'k'] = mat['Thermal Conductivity']
            agg_surface.loc[1,'rho'] = mat['Density']
            agg_surface.loc[1,'cp'] = mat['Specific Heat']

        else:
            for layer in layer1:
                
                d_list.append(surface[f'{roofwall}{layer}Thickness'])
                mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer}Material']]
                k_list.append(mat['Thermal Conductivity'])
                rho_list.append(mat['Density'])
                cp_list.append(mat['Specific Heat'])

                # print('Outer layer [1]: 1 -',layer)
                agg_surface.loc[1,'dz'] = sum(d_list)
                agg_surface.loc[1,'k'] = sum(d_list) / sum(d / k for d, k in zip(d_list, k_list))
                agg_surface.loc[1,'rho'] = sum(d * k for d, k in zip(d_list, rho_list)) / sum(d_list)
                agg_surface.loc[1,'cp'] = sum(d * rho * cp for d, rho, cp in zip(d_list, rho_list, cp_list)) / (agg_surface.loc[1,'rho'] * agg_surface.loc[1,'dz'])

        # ------------------------------------- Layer 2 -------------------------------------
        #                                   Insulation Layer
        # -----------------------------------------------------------------------------------


        # Just take the values in the layer
        mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer2 }Material']]
        agg_surface.loc[2,'dz'] = surface[f'{roofwall}{layer2}Thickness']
        agg_surface.loc[2,'k'] = mat['Thermal Conductivity']
        agg_surface.loc[2,'rho'] = mat['Density']
        agg_surface.loc[2,'cp'] = mat['Specific Heat']

        # ------------------------------------- Layer 3 -------------------------------------
        #                                     Inner Layer
        # -----------------------------------------------------------------------------------
        
        if len(layer3) > 0:

            if len(layer3) == 1:
                print('L3, only one 3 layer')
                mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer3[0]}Material']]
                agg_surface.loc[3,'dz'] = surface[f'{roofwall}{layer3[0]}Thickness']
                agg_surface.loc[3,'k'] = mat['Thermal Conductivity']
                agg_surface.loc[3,'rho'] = mat['Density']
                agg_surface.loc[3,'cp'] = mat['Specific Heat']

                # Fill 2 inner layers with nan
                for layer in range(4,6):
                    agg_surface.loc[layer,'dz'] = nan
                    agg_surface.loc[layer,'k'] = nan
                    agg_surface.loc[layer,'rho'] = nan
                    agg_surface.loc[layer,'cp'] = nan

            else:
                print('L3, more than one 3 layer')

                for layer in layer3:
                    d_list = []
                    k_list = []
                    rho_list = []
                    cp_list = []

                    for layer in layer3:
                        d_list.append(surface[f'{roofwall}{layer}Thickness'])
                        mat = db_dict['Spartacus Material'].loc[surface[f'{roofwall}{layer}Material']]
                        k_list.append(mat['Thermal Conductivity'])
                        rho_list.append(mat['Density'])
                        cp_list.append(mat['Specific Heat'])

                        agg_surface.loc[3,'dz'] = sum(d_list)
                        agg_surface.loc[3,'k'] = sum(d_list) / sum(d / k for d, k in zip(d_list, k_list))
                        agg_surface.loc[3,'rho'] = sum(d * k for d, k in zip(d_list, rho_list)) / sum(d_list)
                        agg_surface.loc[3,'cp'] = sum(d * rho * cp for d, rho, cp in zip(d_list, rho_list, cp_list)) / (agg_surface.loc[3,'rho'] * agg_surface.loc[3,'dz'])

                    # Fill 2 inner layers with nan
                    for layer in range(4,6):
                        agg_surface.loc[layer,'dz'] = nan
                        agg_surface.loc[layer,'k'] = nan
                        agg_surface.loc[layer,'rho'] = nan
                        agg_surface.loc[layer,'cp'] = nan

        else:
            for layer in range(3,6):
                agg_surface.loc[layer,'dz'] = nan
                agg_surface.loc[layer,'k'] = nan
                agg_surface.loc[layer,'rho'] = nan
                agg_surface.loc[layer,'cp'] = nan

        # -----------------------------------------------------------------------------------

    if no_rho == True:
        agg_surface = agg_surface.drop(columns = ['rho'])
        agg_surface = agg_surface.round(3).loc[:,['dz','k','cp']].to_dict()

    else:
        agg_surface = agg_surface.round(3).loc[:,['dz','k','cp','rho']].to_dict()

    agg_surface = {key: list(value.values()) for key, value in agg_surface.items()}
    agg_surface = {key: {'value': value} for key, value in agg_surface.items()}

    return agg_surface

In [ ]:
2e6

2000000.0